<a href="https://colab.research.google.com/github/firojahmed1313/MlAITR/blob/main/pySpark/salesanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").appName("Salesanalysis").getOrCreate()
spark

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,DateType, FloatType

In [7]:
schema=StructType([
  StructField("product_id",IntegerType(),True),
  StructField("customer_id",StringType(),True),
  StructField("order_date",DateType(),True),
  StructField("country",StringType(),True),
  StructField("order_source",StringType(),True)
])

In [8]:
df = spark.read.csv("/content/sample_data/sales.csv", header=False, sep=",", schema=schema)
df.show()
df.printSchema()

+----------+-----------+----------+-------+------------+
|product_id|customer_id|order_date|country|order_source|
+----------+-----------+----------+-------+------------+
|         1|          A|2023-01-01|  India|      Swiggy|
|         2|          A|2022-01-01|  India|      Swiggy|
|         2|          A|2023-01-07|  India|      Swiggy|
|         3|          A|2023-01-10|  India|  Restaurant|
|         3|          A|2022-01-11|  India|      Swiggy|
|         3|          A|2023-01-11|  India|  Restaurant|
|         2|          B|2022-02-01|  India|      Swiggy|
|         2|          B|2023-01-02|  India|      Swiggy|
|         1|          B|2023-01-04|  India|  Restaurant|
|         1|          B|2023-02-11|  India|      Swiggy|
|         3|          B|2023-01-16|  India|      zomato|
|         3|          B|2022-02-01|  India|      zomato|
|         3|          C|2023-01-01|  India|      zomato|
|         1|          C|2023-01-01|     UK|      Swiggy|
|         6|          C|2022-01

In [9]:
schemaMenu=StructType([
  StructField("product_id",IntegerType(),True),
  StructField("product_name",StringType(),True),
  StructField("price",FloatType(),True)
])

In [10]:
dfMenu=spark.read.format("csv").schema(schemaMenu).load("/content/sample_data/menu.csv")
dfMenu.show()
dfMenu.printSchema()

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         1|       PIZZA|100.0|
|         2|     Chowmin|150.0|
|         3|    sandwich|120.0|
|         4|        Dosa|110.0|
|         5|     Biryani| 80.0|
|         6|       Pasta|180.0|
+----------+------------+-----+

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: float (nullable = true)



In [11]:
from pyspark.sql.functions import month,year,dayofmonth,quarter

In [12]:
df=df.withColumn("order_mongth",month("order_date"))
df=df.withColumn("order_year",year("order_date"))
df=df.withColumn("order_day",dayofmonth("order_date"))
df=df.withColumn("order_quarter",quarter("order_date"))
df.show()

+----------+-----------+----------+-------+------------+------------+----------+---------+-------------+
|product_id|customer_id|order_date|country|order_source|order_mongth|order_year|order_day|order_quarter|
+----------+-----------+----------+-------+------------+------------+----------+---------+-------------+
|         1|          A|2023-01-01|  India|      Swiggy|           1|      2023|        1|            1|
|         2|          A|2022-01-01|  India|      Swiggy|           1|      2022|        1|            1|
|         2|          A|2023-01-07|  India|      Swiggy|           1|      2023|        7|            1|
|         3|          A|2023-01-10|  India|  Restaurant|           1|      2023|       10|            1|
|         3|          A|2022-01-11|  India|      Swiggy|           1|      2022|       11|            1|
|         3|          A|2023-01-11|  India|  Restaurant|           1|      2023|       11|            1|
|         2|          B|2022-02-01|  India|      Swiggy

In [13]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").show()

+----------+-----------+----------+-------+------------+------------+----------+---------+-------------+----------+------------+-----+
|product_id|customer_id|order_date|country|order_source|order_mongth|order_year|order_day|order_quarter|product_id|product_name|price|
+----------+-----------+----------+-------+------------+------------+----------+---------+-------------+----------+------------+-----+
|         1|          A|2023-01-01|  India|      Swiggy|           1|      2023|        1|            1|         1|       PIZZA|100.0|
|         2|          A|2022-01-01|  India|      Swiggy|           1|      2022|        1|            1|         2|     Chowmin|150.0|
|         2|          A|2023-01-07|  India|      Swiggy|           1|      2023|        7|            1|         2|     Chowmin|150.0|
|         3|          A|2023-01-10|  India|  Restaurant|           1|      2023|       10|            1|         3|    sandwich|120.0|
|         3|          A|2022-01-11|  India|      Swiggy

***Total Amount spent by each customer***

In [14]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").groupBy("customer_id").sum("price").show()

+-----------+----------+
|customer_id|sum(price)|
+-----------+----------+
|          E|    2040.0|
|          B|    4440.0|
|          D|    1200.0|
|          C|    2400.0|
|          A|    4260.0|
+-----------+----------+



***Total Amount spent by each food category***

In [21]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").groupBy("product_name").sum("price").show()

+------------+----------+
|product_name|sum(price)|
+------------+----------+
|       Pasta|    1080.0|
|       PIZZA|    2100.0|
|    sandwich|    5760.0|
|     Biryani|     480.0|
|     Chowmin|    3600.0|
|        Dosa|    1320.0|
+------------+----------+



***Total Amount of sales in each month***

In [15]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").groupBy("order_mongth").sum("price").show()

+------------+----------+
|order_mongth|sum(price)|
+------------+----------+
|           1|    2960.0|
|           6|    2960.0|
|           3|     910.0|
|           5|    2960.0|
|           7|     910.0|
|          11|     910.0|
|           2|    2730.0|
+------------+----------+



***Yearly sales***

In [16]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").groupBy("order_year").sum("price").show()

+----------+----------+
|order_year|sum(price)|
+----------+----------+
|      2023|    9990.0|
|      2022|    4350.0|
+----------+----------+



***Quaterly sales***

In [17]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").groupBy("order_quarter").sum("price").show()

+-------------+----------+
|order_quarter|sum(price)|
+-------------+----------+
|            1|    6600.0|
|            3|     910.0|
|            4|     910.0|
|            2|    5920.0|
+-------------+----------+



***Total number of order by each category***

In [18]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").groupBy("order_source").sum("price").show()

+------------+----------+
|order_source|sum(price)|
+------------+----------+
|      zomato|    4920.0|
|      Swiggy|    6330.0|
|  Restaurant|    3090.0|
+------------+----------+



***Top 5 ordered items***

In [24]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").groupBy("product_name").count().orderBy("count",ascending=False).limit(5).show()

+------------+-----+
|product_name|count|
+------------+-----+
|    sandwich|   48|
|     Chowmin|   24|
|       PIZZA|   21|
|        Dosa|   12|
|       Pasta|    6|
+------------+-----+



***Top ordered items***

In [25]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").groupBy("product_name").count().orderBy("count",ascending=False).limit(1).show()

+------------+-----+
|product_name|count|
+------------+-----+
|    sandwich|   48|
+------------+-----+



***Frequecy of customer visited in Restaurant***

In [29]:
from pyspark.sql.functions import count_distinct
df.filter(df.order_source=="Restaurant").groupBy("customer_id").agg(count_distinct("order_date")).show()

+-----------+--------------------------+
|customer_id|count(DISTINCT order_date)|
+-----------+--------------------------+
|          E|                         5|
|          B|                         6|
|          D|                         1|
|          C|                         3|
|          A|                         6|
+-----------+--------------------------+



***Total sales by each country***

In [19]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").groupBy("country").sum("price").show()

+-------+----------+
|country|sum(price)|
+-------+----------+
|  India|    4860.0|
|    USA|    2460.0|
|     UK|    7020.0|
+-------+----------+



***Total sales by order source***

In [20]:
df.join(dfMenu, df.product_id==dfMenu.product_id,"leftouter").groupBy("order_source").sum("price").show()

+------------+----------+
|order_source|sum(price)|
+------------+----------+
|      zomato|    4920.0|
|      Swiggy|    6330.0|
|  Restaurant|    3090.0|
+------------+----------+

